In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


# Data processing

In [ ]:
# Tải dữ liệu lịch sử của cổ phiếu (ví dụ: 'META' cho Meta)
ticker = 'META'
data = yf.download(ticker, period='5y')
# Tính giá trung bình
data['Price'] = data[['Open', 'Close', 'High', 'Low']].mean(axis=1)

# Chọn các cột cần thiết
data = data[['Open', 'High', 'Low', 'Close', 'Volume', 'Price']]
data

In [ ]:
# Kiểm tra giá trị bị thiếu
print("Số lượng điểm dữ liệu bị thiếu là")
print(data.isna().sum())

# Xử lý giá trị bị thiếu (loại bỏ các hàng có giá trị bị thiếu)
data.dropna(inplace=True)

# Lưu dữ liệu vào tệp CSV
csv_file = f'{ticker}_historical_data.csv'
data.to_csv(csv_file)
print(f"Dữ liệu đã được lưu vào tệp {csv_file}")

# Đọc dữ liệu từ tệp CSV (Tuỳ chọn)
data_from_csv = pd.read_csv(csv_file, index_col=0)
print(data_from_csv.head())

# Chuẩn bị dữ liệu cho mô hình
prices = data_from_csv['Price'].values
volumes = data_from_csv['Volume'].values

# Split data into train and test sets
split_index = int(0.8 * len(prices)) # 80% of data for training, 20% for testing
train_prices, test_prices = prices[:split_index], prices[split_index:] # Train value contain 80% of the data, test is the 20% remaining
train_volumes, test_volumes = volumes[:split_index], volumes[split_index:] # Train value contain 80% of the data, test is the 20% remaining
train_data, test_data = data[:split_index], data[split_index:] # Train value contain 80% of the data, test is the 20% remaining




In [ ]:
# Căn chỉnh dữ liệu
train_volumes = train_volumes[1:]  # Loại bỏ phần tử đầu tiên của volumes
train_prices = train_prices[:-1]   # Loại bỏ phần tử cuối cùng của prices
test_volumes = test_volumes[1:]  # Loại bỏ phần tử đầu tiên của volumes
test_prices = test_prices[:-1]  # Loại bỏ phần tử cuối cùng của prices

# Gradient Descent

In [ ]:
import random

def predict (prices_for_predict, volumes_for_predict) :

    # Chuẩn hóa dữ liệu
    mean_price = np.mean(prices_for_predict) # Tính giá trị trung bình 
    std_price = np.std(prices_for_predict) # Tính độ lệch chuẩn
    prices_for_predict = (prices_for_predict - mean_price) / std_price
    
    # Khởi tạo tham số
    b = random.random()
    a = random.random()
    alpha = 0.02  # Learning rate
    epochs = 100000  # Số lần lặp
    # Số lượng mẫu
    m = len(volumes_for_predict)

    # Gradient Descent
    for epoch in range(epochs):
        volumes_pred = b + a * prices_for_predict # Compute predict volume
        error = volumes_pred - volumes_for_predict # Calculate the error
        db = (2/m) * np.sum(error) # Compute gradient
        da = (2/m) * np.sum(error * prices_for_predict) # Compute gradient
        
        # Kiểm tra các giá trị không hợp lệ trước khi cập nhật
        if not np.isnan(db) and not np.isnan(da):
            b -= alpha * db
            a -= alpha * da
        else:
            print(f"Encountered NaN at epoch {epoch}")
            break
    # Tính toán giá trị dự đoán cuối cùng
    volumes_pred = b + a * prices_for_predict
    
    print(f"Hệ số hồi quy (intercept): {b}\nHệ số hồi quy (slope): {a}")
    return (volumes_pred, b, a)



# Tính toán hệ số và metric dựa trên test data

In [ ]:
# Dự đoán volume sử dụng các hệ số vừa tìm được
train_volumes_pred, train_incepter, train_slope = predict(train_prices, train_volumes)


## Tính toán metric

In [ ]:
# Tính tổng bình phương của các phần dư hậu hồi quy (SSR)
ssr = mean_squared_error(train_volumes, train_volumes_pred) * len(train_volumes)
print(f"Tổng bình phương của các phần dư (SSR): {ssr}")

# Sử dụng sklearn để so sánh

In [ ]:
# Chuyển đổi prices thành dạng 2D cho sklearn
train_prices_reshaped = train_prices.reshape(-1, 1)
test_prices_reshaped = test_prices.reshape(-1, 1)

In [ ]:

# Khởi tạo và huấn luyện mô hình LinearRegression của sklearn
model = LinearRegression()
model.fit(train_prices_reshaped, train_volumes)

# Dự đoán khối lượng giao dịch với sklearn
train_volumes_pred_sklearn = model.predict(train_prices_reshaped)

# So sánh hệ số hồi quy
print(f"Hệ số hồi quy (intercept) với sklearn: {model.intercept_}")
print(f"Hệ số hồi quy (slope) với sklearn: {model.coef_[0]}")


# Tính SSR cho mô hình sklearn
ssr_sklearn = mean_squared_error(train_volumes, train_volumes_pred_sklearn) * len(train_volumes)
print(f"Tổng bình phương của các phần dư (SSR) với sklearn: {ssr_sklearn}")



# Độ khác biệt giữa giá trị SSR của mô hình sklearn và mô hình DIY

In [ ]:
ssr_sklearn - ssr

# Vẽ biểu đồ so sánh

In [ ]:
plt.figure(figsize=(10, 6)) # Create a figure
plt.plot(train_data.index[1:], train_volumes, label='Thực tế') # Plot the actual trading volume
plt.plot(train_data.index[1:], train_volumes_pred, label='Dự đoán (tự triển khai)', color='y') # Plot the predicted volume
plt.plot(train_data.index[1:], train_volumes_pred_sklearn, label='Dự đoán (sklearn)', color='r', linestyle='--') # Plot the predicted volume using sklearn model
plt.xlabel('Ngày') # Set label of the x-axis
plt.ylabel('Volume') # Set label of the y-axis
plt.title(f'Dự đoán khối lượng giao dịch cho {ticker}') # Set the title
plt.legend() # Add a legend
plt.show() # Display the figure

# Evaluation

In [ ]:

# Calculating evaluation metrics
def adjusted_r2(r2, n, p):
    return 1 - (1 - r2) * ((n - 1) / (n - p - 1))

# Evalute metrics dictionary
metrics = {
    "Model": [],
    "R-Squared": [],
    "Multiple R": [],
    "Standard Error": [],
    "MSE": []
}

# Evaluate model function
def evaluate_model(y_true, y_pred, model_name):
    r2 = r2_score(y_true, y_pred) # measures the proportion
    mse = mean_squared_error(y_true, y_pred) # measures the average squared difference between the predicted and actual values
    std_error = np.sqrt(mse) # Standard error of the model predictions, calculated as the square root of MSE
    metrics["Model"].append(model_name)
    metrics["R-Squared"].append(r2)
    metrics["Multiple R"].append(np.sqrt(r2))
    metrics["Standard Error"].append(std_error)
    metrics["MSE"].append(mse)

model.fit(test_prices_reshaped, test_volumes) # Fit the model
test_volumes_pred, test_b, test_a = predict(test_prices, test_volumes) # Predict with function
test_volumes_pred_sklearn = model.predict(test_prices_reshaped) # Predict with sklearn model

# Evaluate model
evaluate_model(train_volumes, train_volumes_pred, "Custom Least Squares")
evaluate_model(train_volumes, train_volumes_pred_sklearn, "Sklearn LinearRegression")

# Display evaluation metrics
evaluation_df = pd.DataFrame(metrics) # Create DataFrame
print(evaluation_df)# Print DataFrame

plt.figure(figsize=(10, 6)) # Create a figure
plt.plot(test_data.index[1:], test_volumes, label='Thực tế') # Plot the actual volume
plt.plot(test_data.index[1:], test_volumes_pred, label='Dự đoán (tự triển khai)', color='r') # Plot the predicted volume
plt.plot(test_data.index[1:], test_volumes_pred_sklearn, label='Dự đoán (sklearn)', color='g', linestyle='--') # Plot the predicted volume using sklearn model
plt.xlabel('Ngày') # Set x-axis label
plt.ylabel('Volume') # Set y-axis label
plt.title(f'Dự đoán khối lượng giao dịch cho {ticker}') # Set title
plt.legend() # Create a legend
plt.show() # Display the figure